# Import from Matrix Market

Download and convert the matrix to our dense format. 
We may have problems in the future as the matrix can be very large.

In [23]:
import os
import numpy as np

In [19]:
DATASET_NAME = "DatasetXXX_PLEASE CHANGE ME"

link_of_matrix = (
    "https://suitesparse-collection-website.herokuapp.com/MM/HB/1138_bus.tar.gz"
)
matrix_name = link_of_matrix.split("/")[-1]
matrix_market_folder = (
    "/scratch/eschreib/matrix_market_matrices"
)
own_matrices_folder = (
    "/scratch/eschreib/matrices/" + DATASET_NAME
)

# Download the matrix
os.system("wget -P " + matrix_market_folder + " " + link_of_matrix)

print(
    f"Size of the file: {os.path.getsize(os.path.join(matrix_market_folder, matrix_name))} bytes"
)

# Unzip the matrix into the same folder
os.system(
    "tar -xf "
    + os.path.join(matrix_market_folder, matrix_name)
    + " -C "
    + matrix_market_folder
)

matrix_path = os.path.join(
    matrix_market_folder, matrix_name.split(".")[0], matrix_name.split(".")[0] + ".mtx"
)

--2023-11-23 13:59:47--  https://suitesparse-collection-website.herokuapp.com/MM/HB/1138_bus.tar.gz
Resolving suitesparse-collection-website.herokuapp.com (suitesparse-collection-website.herokuapp.com)... 54.208.186.182, 54.243.129.215, 34.201.81.34, ...
Connecting to suitesparse-collection-website.herokuapp.com (suitesparse-collection-website.herokuapp.com)|54.208.186.182|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://sparse-files.engr.tamu.edu/MM/HB/1138_bus.tar.gz [following]
--2023-11-23 13:59:47--  http://sparse-files.engr.tamu.edu/MM/HB/1138_bus.tar.gz
Resolving sparse-files.engr.tamu.edu (sparse-files.engr.tamu.edu)... 52.219.98.144, 52.219.92.104, 52.219.101.116, ...
Connecting to sparse-files.engr.tamu.edu (sparse-files.engr.tamu.edu)|52.219.98.144|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19829 (19K) [application/x-tar]
Saving to: ‘/Users/ericschreiber/dev/ETH/HPC_ETH/project/benchmark_pipeline/

Size of the file: 19829 bytes


. .........                                  100%  155K=0,1s

2023-11-23 13:59:47 (155 KB/s) - ‘/Users/ericschreiber/dev/ETH/HPC_ETH/project/benchmark_pipeline/matrix_market/1138_bus.tar.gz.1’ saved [19829/19829]



In [27]:
def read_file_to_list_of_lists(file_path):
    """The format of the file is as follows:
    1. Lines with % are comments
    2. The first line contains the dimensions of the matrix (num_rows, num_columns, total_num_entries)
    3. The following lines contain the entries of the matrix in the following format:
        row_index column_index value
    """
    with open(file_path, "r") as file:  # returns  tuple(list, int, int, int):
        lines = file.readlines()
        # Remove all comments
        lines = [line for line in lines if not line.startswith("%")]
        # Remove all empty lines
        lines = [line for line in lines if line.strip()]
        # Remove the first line
        num_rows, num_columns, total_num_entries = lines.pop(0).split(" ")

        matrix = np.zeros((int(num_rows), int(num_columns)))
        # Remove the newline character
        lines = [line.strip() for line in lines]
        # Split the lines into lists
        lines = [line.split(" ") for line in lines]

        for line in lines:
            matrix[int(line[0]) - 1, int(line[1]) - 1] = float(line[2])

        return matrix, int(num_rows), int(num_columns), int(total_num_entries)

In [33]:
def write_matrix_to_dense_format(matrix, file_path):
    """Safe in dense format:
    First line: num_rows num_columns data_type (f for float)
    Following lines: matrix entries
    """
    with open(file_path, "w") as file:
        file.write(f"{matrix.shape[0]} {matrix.shape[1]} f\n")
        for row in matrix:
            for entry in row:
                file.write(f"{entry} ")
            file.write("\n")

In [35]:
matrix, num_rows, num_columns, total_num_entries = read_file_to_list_of_lists(
    matrix_path
)
print(
    f"Number of rows: {num_rows}, number of columns: {num_columns}, total number of entries: {total_num_entries}"
)
print(f"Matrix shape: {matrix.shape}")

Number of rows: 1138, number of columns: 1138, total number of entries: 2596
Matrix shape: (1138, 1138)


In [34]:
save_path = os.path.join(own_matrices_folder, matrix_name.split(".")[0])
write_matrix_to_dense_format(matrix, save_path)